In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, mean_squared_error
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1. Завантаження власних даних
df = pd.read_csv("boston.csv")
print("\n Дані успішно завантажено!\n")

# 2. Попередня підготовка даних
le = LabelEncoder()
df['CHAS'] = le.fit_transform(df['CHAS'])  # Перетворення категоріальних змінних
scaler = StandardScaler()
df[df.columns] = scaler.fit_transform(df)
print(" Дані підготовлені: категоріальні змінні закодовані, числові масштабовані.\n")

# Визначаємо вхідні ознаки та цільову змінну для класифікації та регресії
X = df.drop(columns=['MEDV'])
y_class = (df['MEDV'] > df['MEDV'].median()).astype(int)  # Бінаризація для класифікації
y_reg = df['MEDV']  # Для регресії

# 3. Розбиття на навчальну та тестову вибірки
X_train, X_test, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X, y_reg, test_size=0.2, random_state=42)
print(" Дані розбиті: 80% - навчальна вибірка, 20% - тестова.\n")

# 4. Побудова k-NN моделей
knn_classifier = KNeighborsClassifier(n_neighbors=5)
knn_classifier.fit(X_train, y_train_class)
print(" Класифікаційна k-NN модель навчена!\n")

knn_regressor = KNeighborsRegressor(n_neighbors=5)
knn_regressor.fit(X_train_reg, y_train_reg)
print(" Прогнозна k-NN модель навчена!\n")

# 5. Прогноз та оцінка якості моделей
# Класифікація
y_pred_class = knn_classifier.predict(X_test)
accuracy = accuracy_score(y_test_class, y_pred_class)
conf_matrix = confusion_matrix(y_test_class, y_pred_class)
print(f" Точність класифікаційної моделі: {accuracy:.5f}\n")
print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_report(y_test_class, y_pred_class))

# Регресія
y_pred_reg = knn_regressor.predict(X_test_reg)
mse = mean_squared_error(y_test_reg, y_pred_reg)
print(f" MSE для прогнозної моделі: {mse:.5f}\n")

# 6. Підбір оптимальних параметрів
print(" Оптимізація моделей...\n")
param_grid = {'n_neighbors': range(1, 21)}

# Класифікація
grid_search_class = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, scoring='accuracy')
grid_search_class.fit(X_train, y_train_class)
print(f" Найкращі параметри для класифікації: {grid_search_class.best_params_}\n")

# Регресія
grid_search_reg = GridSearchCV(KNeighborsRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search_reg.fit(X_train_reg, y_train_reg)
print(f" Найкращі параметри для регресії: {grid_search_reg.best_params_}\n")

# 7. Оцінка оптимальних моделей
# Класифікація
optimized_knn_class = KNeighborsClassifier(**grid_search_class.best_params_)
optimized_knn_class.fit(X_train, y_train_class)
opt_pred_class = optimized_knn_class.predict(X_test)
opt_accuracy = accuracy_score(y_test_class, opt_pred_class)
print(f" Оптимізована точність класифікації: {opt_accuracy:.5f}\n")

# Регресія
optimized_knn_reg = KNeighborsRegressor(**grid_search_reg.best_params_)
optimized_knn_reg.fit(X_train_reg, y_train_reg)
opt_pred_reg = optimized_knn_reg.predict(X_test_reg)
opt_mse = mean_squared_error(y_test_reg, opt_pred_reg)
print(f" Оптимізоване MSE для регресії: {opt_mse:.5f}\n")


 Дані успішно завантажено!

 Дані підготовлені: категоріальні змінні закодовані, числові масштабовані.

 Дані розбиті: 80% - навчальна вибірка, 20% - тестова.

 Класифікаційна k-NN модель навчена!

 Прогнозна k-NN модель навчена!

 Точність класифікаційної моделі: 0.89216

Confusion Matrix:
[[52  8]
 [ 3 39]]

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.87      0.90        60
           1       0.83      0.93      0.88        42

    accuracy                           0.89       102
   macro avg       0.89      0.90      0.89       102
weighted avg       0.90      0.89      0.89       102

 MSE для прогнозної моделі: 0.24470

 Оптимізація моделей...

 Найкращі параметри для класифікації: {'n_neighbors': 9}

 Найкращі параметри для регресії: {'n_neighbors': 2}

 Оптимізована точність класифікації: 0.85294

 Оптимізоване MSE для регресії: 0.19155

